# Create the map of closest camera to each model space pixel

In [ ]:
import os
from glob import glob
import rioxarray as rxr
import xarray as xr
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

# Define paths to cameras and reference DEM
data_folder = '/Users/rdcrlrka/Research/Soo_locks'
out_folder = os.path.join(data_folder, 'inputs')
cam_positions_file = os.path.join(out_folder, 'Camera_Locations.geojson')
refdem_file = os.path.join(out_folder, 'TLS_DTM_cropped_filled.tif')
image_files = sorted(glob(os.path.join(out_folder, 'original_images', '*.tiff')))
print(f'Found {len(image_files)} images.')

In [ ]:
# Load camera positions
cam_positions = gpd.read_file(cam_positions_file)
# add channel column if it doesn't exist
if 'channel' not in cam_positions.keys():
    # remap camera labels to channels
    remap_dict = {
        'Camera_24': 1,
        'Camera_23': 2,
        'Camera_22': 3,
        'Camera_21': 4,
        'Camera_20': 17,
        'Camera_19': 5,
        'Camera_18': 6,
        'Camera_17': 18,
        'Camera_16': 7,
        'Camera_15': 8,
        'Camera_14': 19,
        'Camera_13': 9,
        'Camera_12': 10,
        'Camera_11': 20,
        'Camera_10': 11,
        'Camera_9': 12,
        'Camera_8': 21,
        'Camera_7': 13,
        'Camera_6': 14,
        'Camera_5': 22,
        'Camera_4': 15,
        'Camera_3': 16,
        'Camera_2': 24,
        'Camera_1': 23

    }
    cam_positions['channel'] = 0
    for i,row in cam_positions.iterrows():
        cam = row['name']
        cam_positions.loc[cam_positions['name']==cam, 'channel'] = remap_dict[cam]
    cam_positions = cam_positions.sort_values(by='channel').reset_index(drop=True)

    # save to file
    cam_positions.to_file(cam_positions_file, index=False)
    print('Camera positions re-saved with with respective channel assignment.')

cam_positions = cam_positions.to_crs("EPSG:32619")
cam_positions['X'] = [x.coords.xy[0][0] for x in cam_positions['geometry']]
cam_positions['Y'] = [x.coords.xy[1][0] for x in cam_positions['geometry']]
cam_positions['Z'] = [x.z for x in cam_positions['geometry']]
cam_centers = cam_positions[['X', 'Y', 'Z']].values


# Load DEM
refdem = rxr.open_rasterio(refdem_file).squeeze()
refdem = xr.where(refdem < -1e3, np.nan, refdem)
refdem = refdem.rio.write_crs("EPSG:32619")

# Plot
fig, ax = plt.subplots(figsize=(6,12))
refdem.plot(ax=ax, cmap='Grays_r')
cam_positions.plot(c=cam_positions['channel'], ax=ax)
ax.grid()
plt.show()

In [ ]:
cam_positions

In [ ]:
# Create 2D arrays of pixel coordinates
dem_x, dem_y = np.meshgrid(refdem['x'].values, refdem['y'].values)

# Flatten for easier distance calculation
pixels = np.column_stack([dem_x.ravel(), dem_y.ravel()])

# Get camera coordinates
cam_coords = np.array([[pt.x, pt.y] for pt in cam_positions.geometry])

# Calculate distance to each camera
# Distance: sqrt((x1-x2)^2 + (y1-y2)^2)
diff = pixels[:, None, :] - cam_coords[None, :, :]
distances = np.linalg.norm(diff, axis=2) 

# Find closest camera for each pixel
closest_cam_idx = np.argmin(distances, axis=1)

# Reshape to DEM shape
closest_cam_map = closest_cam_idx.reshape(refdem.shape)

# Convert to xarray.DataArray
closest_cam_da = xr.DataArray(
    closest_cam_map,
    coords=refdem.coords,
    dims=refdem.dims,
    name="closest_camera"
)

# Mask where DEM is NaN
closest_cam_da = closest_cam_da.astype(float)
closest_cam_da = xr.where(np.isnan(refdem), np.nan, closest_cam_da)

# Plot
cmap = matplotlib.colors.ListedColormap(plt.get_cmap('tab20').colors + plt.get_cmap('tab20b').colors[:4])
bounds = np.arange(0, len(cam_positions)+1)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
camera_labels = np.arange(1, len(cam_positions) + 1)
fig, ax = plt.subplots(figsize=(8,12))
im = ax.imshow(
    closest_cam_da, cmap=cmap, norm=norm,
    extent=(closest_cam_da.x.min()/1e3, closest_cam_da.x.max()/1e3,
            closest_cam_da.y.min()/1e3, closest_cam_da.y.max()/1e3)
)
cbar = fig.colorbar(im, ax=ax, shrink=0.5, ticks=np.arange(len(cam_positions))+0.5)
cbar.ax.set_yticklabels(camera_labels)
cbar.set_label('Closest camera')
ax.set_xlabel('Easting [km]')
ax.set_ylabel('Northing [km]')
plt.show()

# Save outputs
closest_cam_file = os.path.join(out_folder, 'closest_camera_map.tif')
closest_cam_da = xr.where(np.isnan(closest_cam_da), -9999, closest_cam_da)
closest_cam_da = closest_cam_da.astype(np.int16)
closest_cam_da = closest_cam_da.rio.write_nodata(-9999)
closest_cam_da = closest_cam_da.rio.write_crs("EPSG:32619")
closest_cam_da.rio.to_raster(closest_cam_file)
print(f'Closest camera map saved to:\n{closest_cam_file}')

fig_file = os.path.join(out_folder, 'closest_camera_map.png')
fig.savefig(fig_file, dpi=300, bbox_inches='tight')
print(f'Figure saved to:\n{fig_file}')